# NFL API and Prediction Charts for Quiniela

## Config

In [107]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import nflapi
from datetime import datetime

from dash import dash_table
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

## Read in Data

In [109]:
nfl = nflapi.NFL(ua="nflapi")

In [110]:
preds = pd.read_csv('../data/quiniela_res.csv').sort_values(['season', 'week'], ascending=True)
preds['season'] = preds['season'].astype(int)
preds['week'] = preds['week'].astype(int)
preds = preds.rename(columns={'Home Team': 'home_team', 'Away Team':'away_team'})

teamsL = np.unique(preds['away_team'].unique().tolist() + preds['home_team'].unique().tolist())
print(np.sort(teamsL))
print(len(teamsL))
preds.head()

['ARI' 'ATL' 'BAL' 'BUF' 'CAR' 'CHI' 'CIN' 'CLE' 'DAL' 'DEN' 'DET' 'GB'
 'HOU' 'IND' 'JAX' 'KC' 'LA' 'LAC' 'LV' 'MIA' 'MIN' 'NE' 'NO' 'NYG' 'NYJ'
 'PHI' 'PIT' 'SEA' 'SF' 'TB' 'TEN' 'WAS']
32


,season,week,away_team,home_team,Gel,Hector,Emilio,Sonny
0,2003,1,ATL,DAL,DAL,ATL,DAL,DAL
1,2003,1,NE,BUF,BUF,BUF,NE,BUF
2,2003,1,JAX,CAR,JAX,JAX,JAX,CAR
3,2003,1,DEN,CIN,DEN,DEN,DEN,DEN
4,2003,1,ARI,DET,DET,ARI,DET,ARI


In [111]:
teams = pd.read_csv('../data/nfl_team_info_all.csv', index_col=0)
teams['season'] = teams['season'].astype(int)
team_dec = teams.groupby(['full_name', 'abbreviation']).size().reset_index(drop=False).set_index(['full_name'])['abbreviation'].to_dict()
teams.head()

,abbreviation,full_name,nick_name,conference,division,city_state_region,season,venue,logo
0,WAS,Washington Redskins,Redskins,NFC,NFC_EASTERN,Washington,2003,NaN,WAS_logo_old.png
1,TB,Tampa Bay Buccaneers,Buccaneers,NFC,NFC_SOUTH,Tampa Bay,2003,NaN,TB_logo_old.png
2,SEA,Seattle Seahawks,Seahawks,NFC,NFC_WESTERN,Seattle,2003,NaN,SEA_logo_old.png
3,SF,San Francisco 49ers,49ers,NFC,NFC_WESTERN,San Francisco,2003,NaN,SF_logo.webp
4,LAC,San Diego Chargers,Chargers,AFC,AFC_WESTERN,San Diego,2003,NaN,LAC_logo.webp


In [123]:
scores = pd.read_csv('../data/nfl_scores_all.csv', index_col=0)
scores['season'] = scores['season'].astype(int)
scores['week'] = scores['week'].astype(int)
scores['home_team'] = scores['home_team'].apply(lambda x: team_dec[x])
scores['away_team'] = scores['away_team'].apply(lambda x: team_dec[x])
scores['winner'] = [home_team if home_score + home_ot_score > away_score + away_ot_score else \
                    away_team if away_score + away_ot_score > home_score + home_ot_score else \
                    'TIE' for home_ot_score, away_ot_score, home_score, away_score, home_team, away_team in \
                    scores[['home_team_score', 'away_team_score', 
                            'home_points_overtime_total', 'visitor_points_overtime_total', 
                            'home_team', 'away_team']].values]
scores['total_home_score'] = scores['home_team_score'] + scores['home_points_overtime_total']
scores['total_away_score'] = scores['away_team_score'] + scores['visitor_points_overtime_total']
scores['home_total_yards'] = scores['home_passing_yards'] + scores['home_rushing_yards']
scores['away_total_yards'] = scores['visitor_passing_yards'] + scores['visitor_rushing_yards']

scores.head()

,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,visitor_passing_yards,visitor_passing_touchdowns,visitor_rushing_yards,visitor_rushing_touchdowns,phase,winner,total_home_score,total_away_score,home_total_yards,away_total_yards
0,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,WAS,NYJ,16.0,...,101.0,0.0,57.0,1.0,FINAL,WAS,16.0,13.0,327.0,158.0
1,2003,1,REG,10012003-0907-0075-30bd-8e1167d1c6ac,10160000-0264-997c-6c87-72c644130bbf,2003-09-07 09:00:00+00:00,2003-09-07,DET,ARI,42.0,...,339.0,3.0,95.0,0.0,FINAL,DET,42.0,24.0,261.0,434.0
2,2003,1,REG,10012003-0907-0126-f566-a59e19a70a3a,10160000-0264-97a8-19af-9168b28ada0a,2003-09-07 09:00:00+00:00,2003-09-07,CIN,DEN,10.0,...,109.0,0.0,184.0,2.0,FINAL,DEN,10.0,30.0,294.0,293.0
3,2003,1,REG,10012003-0907-0277-e7c6-41b36cc4a1ce,10160000-0264-9839-6579-47a20841e644,2003-09-07 09:00:00+00:00,2003-09-07,CLE,IND,6.0,...,204.0,0.0,67.0,0.0,FINAL,IND,6.0,9.0,280.0,271.0
4,2003,1,REG,10012003-0907-0388-64cf-dd9886d18dde,10160000-0265-04d2-fd24-f2f5b0964d49,2003-09-07 09:00:00+00:00,2003-09-07,PIT,BAL,34.0,...,143.0,1.0,88.0,1.0,FINAL,PIT,34.0,15.0,339.0,231.0


In [124]:
## Merge Home Team Info
df = scores.merge(teams.rename(columns={x:x+'_home' for x in teams.columns if x != 'season'}),
                    left_on=['home_team', 'season'], 
                    right_on=['abbreviation_home', 'season'], 
                    suffixes=['', '_home'],
                    how='left')

## Merge Away Team Info
df = df.merge(teams.rename(columns={x:x+'_away' for x in teams.columns if x != 'season'}), 
                    left_on=['away_team', 'season'], 
                    right_on=['abbreviation_away', 'season'], 
                    suffixes=['', '_away'],
                    how='left')

## Create New Week Num Column
week_type_order = {'REG':1, 'WC':2, 'DIV':3, 'CONF':4, 'SB':5}
df['week_type_ord'] = df['week_type'].apply(lambda x: week_type_order[x])
week_dec = df.sort_values(['game_date', 'week_type_ord', 'season'], ascending=True).reset_index(drop=True).groupby(['season', 'week_type_ord', 'week']).size().reset_index(drop=False)
week_dec['week_num'] = week_dec.groupby(['season'])['season'].cumcount()+1
df = df.merge(week_dec.drop(0, axis=1), 
                    left_on=['season', 'week', 'week_type_ord'],
                    right_on=['season', 'week', 'week_type_ord'],
                    how='left')


## Merge Users Pred Info
# df = df.merge(preds.drop(['week'], axis=1), 
df = df.merge(preds, 
                    left_on=['home_team', 'away_team', 'season'], 
                    right_on=['home_team', 'away_team', 'season'],
                    how='left',
                    suffixes=['', '_quin'])

## Remove Duplicate Entries
dup_games = scores.groupby(['home_team', 'away_team', 'season']).size()
dup_games = dup_games[dup_games>1].reset_index(drop=False)
drop_idxL = []
for idx, dup_game in dup_games.iterrows():
    temp_dup = df.loc[((df['home_team'] == dup_game['home_team']) & (df['away_team'] == dup_game['away_team']) &  (df['season'] == dup_game['season']))].reset_index(drop=False)
    temp_dup['week_num_diff'] = abs(temp_dup['week_quin'] - temp_dup['week_num'])
    drop_idxL = drop_idxL + temp_dup.sort_values(['week_num_diff'], ascending=False).drop_duplicates(['game_id'], keep='first')['index'].values.tolist()
df = df.loc[~df.index.isin(drop_idxL)].reset_index(drop=True)

## Add Other Cols
## Get Total Score
df['total_home_score'] = df['home_team_score'] + df['home_points_overtime_total']
df['total_away_score'] = df['away_team_score'] + df['visitor_points_overtime_total']

## Preview Data
print(df.columns.tolist())
df.head()

['season', 'week', 'week_type', 'game_id', 'game_details_id', 'game_time', 'game_date', 'home_team', 'away_team', 'home_team_score', 'away_team_score', 'home_points_overtime_total', 'visitor_points_overtime_total', 'home_points_q1', 'home_points_q2', 'home_points_q3', 'home_points_q4', 'home_passing_yards', 'home_passing_touchdowns', 'home_rushing_yards', 'home_rushing_touchdowns', 'visitor_points_q1', 'visitor_points_q2', 'visitor_points_q3', 'visitor_points_q4', 'visitor_passing_yards', 'visitor_passing_touchdowns', 'visitor_rushing_yards', 'visitor_rushing_touchdowns', 'phase', 'winner', 'total_home_score', 'total_away_score', 'home_total_yards', 'away_total_yards', 'abbreviation_home', 'full_name_home', 'nick_name_home', 'conference_home', 'division_home', 'city_state_region_home', 'venue_home', 'logo_home', 'abbreviation_away', 'full_name_away', 'nick_name_away', 'conference_away', 'division_away', 'city_state_region_away', 'venue_away', 'logo_away', 'week_type_ord', 'week_num', '

,season,week,week_type,game_id,game_details_id,game_time,game_date,home_team,away_team,home_team_score,...,city_state_region_away,venue_away,logo_away,week_type_ord,week_num,week_quin,Gel,Hector,Emilio,Sonny
0,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,WAS,NYJ,16.0,...,New York Jets,NaN,NYJ_logo_old.png,1,1,1.0,WAS,WAS,NYJ,WAS
1,2003,1,REG,10012003-0907-0075-30bd-8e1167d1c6ac,10160000-0264-997c-6c87-72c644130bbf,2003-09-07 09:00:00+00:00,2003-09-07,DET,ARI,42.0,...,Arizona,NaN,ARI_logo.webp,1,1,1.0,DET,ARI,DET,ARI
2,2003,1,REG,10012003-0907-0126-f566-a59e19a70a3a,10160000-0264-97a8-19af-9168b28ada0a,2003-09-07 09:00:00+00:00,2003-09-07,CIN,DEN,10.0,...,Denver,NaN,DEN_logo.webp,1,1,1.0,DEN,DEN,DEN,DEN
3,2003,1,REG,10012003-0907-0277-e7c6-41b36cc4a1ce,10160000-0264-9839-6579-47a20841e644,2003-09-07 09:00:00+00:00,2003-09-07,CLE,IND,6.0,...,Indianapolis,NaN,IND_logo.webp,1,1,1.0,IND,CLE,IND,CLE
4,2003,1,REG,10012003-0907-0388-64cf-dd9886d18dde,10160000-0265-04d2-fd24-f2f5b0964d49,2003-09-07 09:00:00+00:00,2003-09-07,PIT,BAL,34.0,...,Baltimore,NaN,BAL_logo.webp,1,1,1.0,PIT,PIT,PIT,PIT


In [125]:
df.to_csv('../data/full_data.csv', index=False)

## Team Statistics

In [126]:
HOME_COLS = {}
HOME_COLS['id'] = ['home_team', 'abbreviation_home', 'nick_name_home',
       'full_name_home', 'conference_home', 'division_home', 'city_state_region_home', 'venue_home','logo_home', ]
HOME_COLS['val_for'] =  ['home_points_q1', 'home_points_q2',
       'home_points_q3', 'home_points_q4', 'home_points_overtime_total', 'home_team_score', 'home_passing_yards',
       'home_passing_touchdowns', 'home_rushing_yards',
       'home_rushing_touchdowns', 'home_total_score', 'home_total_yards']

In [127]:
AWAY_COLS = {}
AWAY_COLS['id'] = ['away_team', 'abbreviation_away','nick_name_away',
       'full_name_away', 'conference_away', 'division_away', 'city_state_region_away', 'venue_away','logo_away',]
AWAY_COLS['val'] = ['visitor_points_q1', 'visitor_points_q2',
       'visitor_points_q3', 'visitor_points_q4', 'visitor_points_overtime_total',  'away_team_score', 
       'visitor_passing_yards', 'visitor_passing_touchdowns',
        'visitor_rushing_yards','visitor_rushing_touchdowns', 'away_total_score', 'away_total_yards']

In [213]:
df_home = df.copy()

df_home = df_home.drop([x for x in AWAY_COLS['id'] if x != 'away_team'][0], axis=1)
df_home = df_home.rename(columns={x:x.replace('visitor', 'against').replace('away', 'against').replace('_home', '').replace('home_', '') for x in df_home.columns})
df_home['home_or_away'] = 'home'

df_away = df.copy()
df_away = df_away.drop([x for x in HOME_COLS['id'] if x != 'home_team'][0], axis=1)
df_away = df_away.rename(columns={x:x.replace('home', 'against').replace('_away', '').replace('away_', '').replace('visitor_', '') \
    for x in df_away.columns})
df_away['home_or_away'] = 'away'

df_teams = pd.concat([df_home, df_away]).sort_values(['season', 'week', 'game_time']).reset_index(drop=True)

## Get Division and Conference Wins
# df_teams['div_winner'] = [1 if ((x==y) & (team == win)) else 0 for x,y,team, win in df_teams[['division', 'division_against', 'team','winner']].values]
# df_teams['conf_winner'] = [1 if ((x==y) & (team == win)) else 0 for x,y,team, win in df_teams[['conference', 'conference_against', 'team', 'winner']].values]
df_teams['div_game'] = [1 if (x==y) else 0 for x,y in df_teams[['division', 'division_against']].values]
df_teams['conf_game'] = [1 if (x==y) else 0 for x,y in df_teams[['conference', 'conference_against']].values]
df_teams.head()


,season,week,week_type,game_id,game_details_id,game_time,game_date,team,against_team,team_score,...,week_type_ord,week_num,week_quin,Gel,Hector,Emilio,Sonny,home_or_away,div_game,conf_game
0,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,WAS,NYJ,16.0,...,1,1,1.0,WAS,WAS,NYJ,WAS,home,0,0
1,2003,1,REG,10012003-0904-00a6-ff62-12e477623f1a,10160000-0264-94ab-68f5-9ff371a986a3,2003-09-04 09:00:00+00:00,2003-09-04,NYJ,WAS,13.0,...,1,1,1.0,WAS,WAS,NYJ,WAS,away,0,0
2,2003,1,REG,10012003-0907-0075-30bd-8e1167d1c6ac,10160000-0264-997c-6c87-72c644130bbf,2003-09-07 09:00:00+00:00,2003-09-07,DET,ARI,42.0,...,1,1,1.0,DET,ARI,DET,ARI,home,0,1
3,2003,1,REG,10012003-0907-0126-f566-a59e19a70a3a,10160000-0264-97a8-19af-9168b28ada0a,2003-09-07 09:00:00+00:00,2003-09-07,CIN,DEN,10.0,...,1,1,1.0,DEN,DEN,DEN,DEN,home,0,1
4,2003,1,REG,10012003-0907-0277-e7c6-41b36cc4a1ce,10160000-0264-9839-6579-47a20841e644,2003-09-07 09:00:00+00:00,2003-09-07,CLE,IND,6.0,...,1,1,1.0,IND,CLE,IND,CLE,home,0,1


In [197]:
def team_stats(team, season=None, week=None, df_teams=df_teams):

    loc = 1
    if week==1:
        season=season-1
        week = df_teams.loc[df_teams['season']==season].week_num.max()
        loc = 0

    team_df = df_teams.loc[df_teams['team']==team].reset_index(drop=True)

    if len(team_df) > 0:
        team_df = team_df.loc[team_df['season']==season].reset_index(drop=True)

    if len(team_df) > 0:
        team_df = team_df.loc[team_df['week_num']<=week].reset_index(drop=True)

    ## Previous Three Games
    last_3_games = team_df.sort_values(['game_time'], ascending=False).iloc[loc:].head(3)[['week_type', 'week_num', 'against_team', 'team', 'team_score', 'against_team_score', 'points_overtime_total','against_points_overtime_total', 'home_or_away']]
    last_3_gamesL = [html.P('Last Games:')]
    for idx, game in last_3_games.iterrows():
        if game['home_or_away'] == 'home':
            last_3_gamesL.append(html.P("{} {} - Week {} ".format(season, game['week_type'], str(game['week_num']))  + \
                game['against_team'] + \
                '\t {} at {} '.format(int(game['against_team_score']+game['against_points_overtime_total']), 
                int(game['team_score']+game['points_overtime_total'])) + game['team']))
        else:
            last_3_gamesL.append(html.P("{} {} - Week {} ".format(season, game['week_type'], str(game['week_num'])) + \
                game['team'] + \
                '\t {} at {} '.format(int(game['team_score']+game['points_overtime_total']), 
                int(game['against_team_score']+game['against_points_overtime_total'])) + game['against_team']))

    ## Win Streak
    last_5_games = team_df.sort_values(['game_time'], ascending=False).iloc[loc:].head(5)[['against_team', 'team', 'winner', 'home_or_away']]
    win_streak = "Streak: "
    for idx, game in last_5_games.iterrows():
        if game['winner'] == game['team']:
            win_streak += 'W'
        else:
            win_streak += 'L'

    ## Off. Yard Rank
    ## Def. Yard Rank

    # return team_df.groupby(['team']).sum().reset_index(drop=False).drop(['season', 'week', 'week_num', 'week_type_ord',
    #                                                                     'week_quin'], axis=1)

    return last_3_gamesL +   [html.P(win_streak)]

res = team_stats(team='WAS', season=2019, week=4, df_teams=df_teams)
res

[P('Last Games:'),
 P('2019 REG - Week 3 CHI\t 31 at 15 WAS'),
 P('2019 REG - Week 2 DAL\t 31 at 21 WAS'),
 P('2019 REG - Week 1 WAS\t 27 at 32 PHI'),
 P('Streak: LLL')]

In [198]:
season=2019
week=5
team='WAS'

loc = 1
if week==1:
    season=season-1
    week = df_teams.loc[df_teams['season']==season].week_num.max()
    loc = 0

team_df = df_teams.loc[df_teams['team']==team].reset_index(drop=True)

if len(team_df) > 0:
    team_df = team_df.loc[team_df['season']==season].reset_index(drop=True)

if len(team_df) > 0:
    team_df = team_df.loc[team_df['week_num']<=week].reset_index(drop=True)

## Previous Three Games
last_3_games = team_df.sort_values(['game_time'], ascending=False).iloc[loc:].head(3)[['week_type', 'week_num', 'against_team', 'team', 'team_score', 'against_team_score', 'points_overtime_total','against_points_overtime_total', 'home_or_away']]
last_3_gamesL = [html.P('Last Games:')]
for idx, game in last_3_games.iterrows():
    if game['home_or_away'] == 'home':
        last_3_gamesL.append(html.P("{} {} - Week {} ".format(season, game['week_type'], str(game['week_num']))  + \
            game['against_team'] + \
            '\t {} at {} '.format(int(game['against_team_score']+game['against_points_overtime_total']), 
            int(game['team_score']+game['points_overtime_total'])) + game['team']))
    else:
        last_3_gamesL.append(html.P("{} {} - Week {} ".format(season, game['week_type'], str(game['week_num'])) + \
            game['team'] + \
            '\t {} at {} '.format(int(game['team_score']+game['points_overtime_total']), 
            int(game['against_team_score']+game['against_points_overtime_total'])) + game['against_team']))

## Win Streak
last_5_games = team_df.sort_values(['game_time'], ascending=False).iloc[loc:].head(5)[['against_team', 'team', 'winner', 'home_or_away']]
win_streak = "Streak: "
for idx, game in last_5_games.iterrows():
    if game['winner'] == game['team']:
        win_streak += 'W'
    else:
        win_streak += 'L'



## Users Statistics

In [199]:
USER_LIST = ['Gel','Hector','Emilio','Sonny']
def user_stats(userL=USER_LIST, season=None, week=None, df=df):

    user_df = df.copy()

    if pd.notnull(season):
        user_df = user_df.loc[user_df['season']==season].reset_index(drop=True)

    if pd.notnull(week):
        user_df = user_df.loc[user_df['week_num']<=week].reset_index(drop=True)

    for user in userL:

        user_df.loc[user_df[user] == user_df['winner'], user+'_correct'] = 1
        user_df.loc[user_df[user] != user_df['winner'], user+'_correct'] = 0

    user_df = user_df.groupby(['season', 'week_num'])[[x+'_correct' for x in userL]].sum()
    user_df[[x+'_score' for x in userL]] = user_df.groupby(['season', 'week_num'])[[x+'_correct' for x in userL]].sum().cumsum()


    return user_df

In [200]:
USER_LIST = ['Gel','Hector','Emilio','Sonny']
userL=USER_LIST

user_df = df.copy()[USER_LIST+['winner', 'season', 'week_num']]

for user in userL:

    user_df.loc[user_df[user] == user_df['winner'], user+'_correct'] = 1
    user_df.loc[user_df[user] != user_df['winner'], user+'_correct'] = 0

user_df = user_df.groupby(['season', 'week_num'])[[x+'_correct' for x in userL]].sum()
user_df[[x+'_score' for x in userL]] = user_df.groupby(['season'])[[x+'_correct' for x in userL]].cumsum()
user_df = user_df.reset_index(drop=False)

In [201]:
season=2019
week=18
curr_user = user_df.loc[((user_df.season==season) & (user_df.week_num==week))]

curr_user = pd.concat([
            curr_user[[x+'_correct' for x in USER_LIST]].rename(columns={x+'_correct':x for x in USER_LIST}).T,
            curr_user[[x+'_score' for x in USER_LIST]].rename(columns={x+'_score':x for x in USER_LIST}).T,
        ], axis=1)
curr_user.columns = ['Week Score', 'Season Score']
curr_user = curr_user.T.reset_index(drop=False)
curr_user = curr_user.rename(columns={'index':''})
curr_user

,,Gel,Hector,Emilio,Sonny
0,Week Score,1.0,1.0,1.0,1.0
1,Season Score,165.0,168.0,159.0,159.0


In [202]:
curr_table = go.Table(
                    header = dict(values=curr_user.columns.tolist()),
                    cells = dict(
                        values= [
                            curr_user[col].astype(int).values.tolist() if col != '' \
                                else curr_user[col].values.tolist() \
                                    for col in curr_user.columns
                        ]
                    ),
            )

fig = go.Figure(data=[curr_table])

In [203]:
[{'label':"{week} - {week_type}".format(week=week, week_type=week_type), 'value':week} for week_num, week, week_type in df.loc[df['season']==preds['season'].max()]\
                            .sort_values('week_num', ascending=True).groupby(['week_num', 'week', 'week_type']).size()\
                                .reset_index(drop=False)[['week_num', 'week', 'week_type']].values]

[{'label': '1 - REG', 'value': 1},
 {'label': '2 - REG', 'value': 2},
 {'label': '3 - REG', 'value': 3},
 {'label': '4 - REG', 'value': 4},
 {'label': '5 - REG', 'value': 5},
 {'label': '6 - REG', 'value': 6},
 {'label': '7 - REG', 'value': 7},
 {'label': '8 - REG', 'value': 8},
 {'label': '9 - REG', 'value': 9},
 {'label': '10 - REG', 'value': 10},
 {'label': '11 - REG', 'value': 11},
 {'label': '12 - REG', 'value': 12},
 {'label': '13 - REG', 'value': 13},
 {'label': '14 - REG', 'value': 14},
 {'label': '15 - REG', 'value': 15},
 {'label': '16 - REG', 'value': 16},
 {'label': '17 - REG', 'value': 17},
 {'label': '18 - REG', 'value': 18},
 {'label': '1 - WC', 'value': 1},
 {'label': '2 - DIV', 'value': 2},
 {'label': '3 - CONF', 'value': 3},
 {'label': '4 - SB', 'value': 4}]

## User Team Stats

In [204]:
user='Emilio'
home_team = 'GB'
away_team = 'TB'
season = 2020
week = 1

def user_team_stats(user, home_team, away_team, season, week):

    from sklearn.metrics import confusion_matrix

    min_date = df_teams.loc[df_teams['season']== season].loc[df_teams['week'] == week].game_date.min()
    curr_teams = df_teams.loc[df_teams['team'].isin([home_team, away_team])].loc[df_teams['game_date']<min_date].reset_index(drop=False)

    user_acc = pd.DataFrame([], columns=['team', 'home', 'total', 'tn', 'fp', 'fn', 'tp', 'acc'])

    count = 0
    for (team, hw), vals in curr_teams.groupby(['team', 'home_or_away'])[[user, 'winner', 'game_date']]:

        vals = vals.sort_values(['game_date']).head(5)

        actual = [x if x == team else 'other' for x in vals['winner']]
        user_pred = [x if x == team else 'other' for x in vals[user]]

        cm = confusion_matrix(actual, user_pred).ravel()

        if len(cm) == 1:
            if np.unique(actual)[0] == 'other':
                tn, fp, fn, tp = len(actual),0,0,0
            else:
                tn, fp, fn, tp = 0,0,0,len(actual)
        else:
            tn, fp, fn, tp = cm

        acc = ""
        for act, pred in zip(actual, user_pred):
            if act == pred:
                acc += 'T'
            else:
                acc += 'F'

        user_acc.loc[count] = [team, hw, len(vals), tn, fp, fn, tp, acc]

        count += 1       

    return user_acc

user_team_stats(user, home_team, away_team, season, week).to_string(index=False).split('\n')

['team home  total  tn  fp  fn  tp   acc',
 '  GB away      5   3   0   2   0 FTFTT',
 '  GB home      5   2   0   3   0 FTTFF',
 '  TB away      5   3   0   2   0 TTTFF',
 '  TB home      5   0   1   2   2 FTFFT']

## Team Stats

In [276]:
df_teams.columns

Index(['season', 'week', 'week_type', 'game_id', 'game_details_id',
       'game_time', 'game_date', 'team', 'against_team', 'team_score',
       'against_team_score', 'points_overtime_total',
       'against_points_overtime_total', 'points_q1', 'points_q2', 'points_q3',
       'points_q4', 'passing_yards', 'passing_touchdowns', 'rushing_yards',
       'rushing_touchdowns', 'against_points_q1', 'against_points_q2',
       'against_points_q3', 'against_points_q4', 'against_passing_yards',
       'against_passing_touchdowns', 'against_rushing_yards',
       'against_rushing_touchdowns', 'phase', 'winner', 'total_score',
       'total_against_score', 'total_yards', 'against_total_yards',
       'abbreviation', 'full_name', 'nick_name', 'conference', 'division',
       'city_state_region', 'venue', 'logo', 'full_name_against',
       'nick_name_against', 'conference_against', 'division_against',
       'city_state_region_against', 'venue_against', 'logo_against',
       'week_type_ord', 'w

In [277]:
def team_rankings(df_teams, week, season):

    ## Get Data For Current Week and Season
    curr_teams = df_teams.loc[df_teams['season']==season].loc[df_teams['week_num']<=week].reset_index(drop=True)

    ## Against Cols
    # against_cols = ['against_passing_yards','against_passing_touchdowns', 'against_rushing_yards','against_rushing_touchdowns', 'total_against_score', 'against_total_yards']
    against_cols = ['total_against_score', 'against_total_yards']

    ## For Cols
    # for_cols = ['passing_yards', 'passing_touchdowns', 'rushing_yards', 'rushing_touchdowns', 'total_score', 'total_yards']
    for_cols = ['total_score', 'total_yards']

    ## Div or Conf Cols
    curr_teams['game'] = 1
    curr_teams['win'] = [1 if x==y else 0 for x,y in curr_teams[['team', 'winner']].values]
    other_cols = ['win', 'game', 'conf_game', 'div_game']

    ## Team Overall Summary
    team_summ = curr_teams.groupby(['team', 'division', 'conference'])[against_cols + for_cols + other_cols].sum().reset_index(drop=False)

    ########## Rankings ###############
    rankings = {}

    ## Division Ranking
    rankings['div'] = pd.concat([
                            team_summ[['team', 'div_game', 'win']], 
                            team_summ.groupby(['division'])[for_cols + ['win']].rank(ascending=False, method='min').rename(columns={'win':'RANK'}),
                            team_summ.groupby(['division'])[against_cols].rank(ascending=True, method='min'), 
                            ], axis=1).rename(columns={'div_game':'game'})
    rankings['div']['SET'] = 'DIV'


    ## Conference Ranking
    rankings['conf'] = pd.concat([
                            team_summ[['team', 'conf_game', 'win']], 
                            team_summ.groupby(['conference'])[for_cols + ['win']].rank(ascending=False, method='min').rename(columns={'win':'RANK'}),
                            team_summ.groupby(['conference'])[against_cols].rank(ascending=True, method='min'), 
                    ], axis=1).rename(columns={'conf_game':'game'})
    rankings['conf']['SET'] = 'CONF'


    ## Overall Ranking
    rankings['all'] = pd.concat([
                            team_summ[['team', 'game', 'win']], 
                            team_summ[for_cols + ['win']].rank(ascending=False, method='min').rename(columns={'win':'RANK'}),
                            team_summ[against_cols].rank(ascending=True, method='min'), 
                            ], axis=1)
    rankings['all']['SET'] = 'ALL'

    ## Concat 
    rankings = pd.concat(rankings.values())
    rankings.columns = ['TEAM', 'GP', 'W', 'OFF-PTS', 'OFF-YRDS','RANK', 'DEF-PTS', 'DEF-YRDS', 'SET']
    for col in ['GP', 'W', 'DEF-PTS', 'DEF-YRDS', 'OFF-PTS', 'OFF-YRDS', 'RANK']:
        rankings[col] = rankings[col].astype(int)
    rankings = rankings[['TEAM', 'SET', 'RANK', 'GP', 'W', 'OFF-PTS', 'OFF-YRDS', 'DEF-PTS', 'DEF-YRDS']]

    ########## AVG YARD AND SCORE ###############
    team_avg = curr_teams.groupby(['team'])[against_cols + for_cols].mean().reset_index(drop=False)
    team_avg.columns = ['TEAM', 'OFF-PTS', 'OFF-YRDS', 'DEF-PTS', 'DEF-YRDS',]

    return rankings, team_avg

In [302]:
rankings, team_summ = team_rankings(df_teams, week=5, season=2019)

In [303]:
team_week_ranking = rankings.loc[rankings['TEAM']==team].drop(['TEAM'], axis=1).set_index(['SET'])
team_week_ranking

,RANK,GP,W,OFF-PTS,OFF-YRDS,DEF-PTS,DEF-YRDS
SET,,,,,,,
DIV,4,3,0,4,4,4,3
CONF,16,4,0,16,15,15,14
ALL,29,5,0,30,28,30,29


In [304]:
team_week_summ = team_summ.loc[team_summ['TEAM']==team].drop(['TEAM'], axis=1)
team_week_summ.index = ['VAL']
team_week_summ = pd.concat([team_week_summ, team_week_ranking[team_week_summ.columns]]).reset_index(drop=False).rename(columns={'index':''})

team_week_summ

,,OFF-PTS,OFF-YRDS,DEF-PTS,DEF-YRDS
0,VAL,30.2,407.8,14.6,281.6
1,DIV,4.0,4.0,4.0,3.0
2,CONF,16.0,15.0,15.0,14.0
3,ALL,30.0,28.0,30.0,29.0
